# Analyze Reservoir Dynamics

In [ ]:
import os
from matplotlib import pyplot as plt
import numpy as np
from preprocess_digits import RECORDINGS_DIR, save_data, load_data
from train import idx_train_test
import pickle

from reservoir import ExperimentalReservoir

In [ ]:
plt.rc("font", family="Arial")
plt.rc("mathtext", fontset="cm")

In [ ]:
def plot_results(dump, twod=False):    
    x = dump["x"]
    y = dump["y"]
    state = dump["states"]
    i = np.argmin(dump["trials"]["nmse"])
    idx_train = dump["trials"]["idx_train"][i]
    idx_test = dump["trials"]["idx_test"][i]

    x_train = np.concatenate([x[_] for _ in idx_train])
    x_test = np.concatenate([x[_] for _ in idx_test])
    y_train = np.concatenate([y[_] for _ in idx_train])
    y_test = np.concatenate([y[_] for _ in idx_test])
    state_train = np.concatenate([state[_] for _ in idx_train])
    state_test = np.concatenate([state[_] for _ in idx_test])
    uin = dump["Win"].dot(x_test.T).T
    readout = dump["trials"]["Wout"][i].T.dot(state_test.T).T + dump["trials"]["bias"][i]
    nmse = dump["trials"]["nmse"][i]
    nrmse = dump["trials"]["nrmse"][i]
    idx_test = dump["trials"]["idx_test"]
    ymax = np.max(y_test)
    ymin = np.min(y_test)

    if not twod:
        fig = plt.figure(figsize=(7, 3))
        ax = fig.subplots()
        ax.plot(x_test.T.ravel(), label="Input Signal")
        ax.plot(y_test.T.ravel(), label="Target")
        ax.plot(readout.T.ravel(), '--', label="Prediction")
        ax.set_xlabel("Timestep")
        ax.set_ylabel("x")
        ax.legend(loc="lower right")
        
        ax.text(0.05, 0.1, "NMSE={:.3f}\nNRMSE={:.3f}".format(nmse, nrmse),
                backgroundcolor="w", transform=ax.transAxes)
    else:
        prediction = np.full_like(readout, -1)
        idx = np.argmax(readout, axis=1)
        for i, _ in enumerate(idx):
            prediction[i, _] = 1
        fig = plt.figure(figsize=(10, 3))
        ax = fig.subplots(1, 4, sharex=True, gridspec_kw=dict(wspace=0.3))
        ax[0].pcolormesh(x_test.T, linewidth=0)
        ax[1].pcolormesh(y_test.T, linewidth=0)
        ax[2].pcolormesh(readout.T, linewidth=0)
        ax[3].pcolormesh(prediction.T, linewidth=0)
        for _ in ax:
            _.set_xlabel("Timestep")
        wer = dump["trials"]["wer"][0]
        ax[3].text(
            0.05, 0.98,
            "NMSE={:.3f}\nWER={:.1f}%".format(nmse, wer * 100),
            va='top',
            color="w", transform=ax[3].transAxes
        )
        np.max(dump["trials"]["rsquare"])
    return fig, ax

In [ ]:
with open("data/hyperopt_sin_square_488kHz_simulation_0.09624870918374559.pickle", "rb") as f:
    dump = pickle.load(f)
    
print("G = {:.2f}".format(dump["hypers"]["optical_power"] / 5.4 * 10))
print("Phi0 = {:.2f} pi".format(dump["hypers"]["phi0"] / np.pi))
print("tau_D = {:.2f} T".format(dump["hypers"]["fir_length"] / float(dump["hypers"]["fir_rate"][:-3]) / (50 / 488e-3)))
print("ridge = {}".format(dump["hypers"]["ridge"]))

fig, ax = plot_results(dump)

In [ ]:
with open("data/hyperopt_narma10_0.561071269333272.pickle", "rb") as f:
    dump = pickle.load(f)

print("G = {:.2f}".format(dump["hypers"]["optical_power"] / 5.4 * 10))
print("Phi0 = {:.2f} pi".format(dump["hypers"]["phi0"] / np.pi))
print("tau_D = {:.2f} T".format(dump["hypers"]["fir_length"] / float(dump["hypers"]["fir_rate"][:-3]) / (50 / 488e-3)))
print("ridge = {}".format(dump["hypers"]["ridge"]))

fig, ax = plot_results(dump)
ax.set_xlim(0, 200)

# fig.savefig("../../Figures/narma10_test.pdf", dpi=300)

In [ ]:
with open("data/hyperopt_japanese_vowels_0.27112363447897525.pickle", "rb") as f:
    dump = pickle.load(f)

fig, ax = plot_results(dump, True)
label_position = (0.01, 0)
ax[0].set_title("a    Input", position=label_position, ha="left")
ax[1].set_title("b    Target", position=label_position, ha="left")
ax[2].set_title("c    Readout", position=label_position, ha="left")
ax[3].set_title("d    Prediction", position=label_position, ha="left")

ax[0].set_ylabel("Cepstrum Coefficient")

print("G = {:.2f}".format(dump["hypers"]["optical_power"] / 5.4 * 10))
print("Phi0 = {:.2f} pi".format(dump["hypers"]["phi0"] / np.pi))
print("tau_D = {:.2f} T".format(dump["hypers"]["fir_length"] / float(dump["hypers"]["fir_rate"][:-3]) / (50 / 488e-3)))
print("ridge = {}".format(dump["hypers"]["ridge"]))

nfeatures = dump["x"][0].shape[1]
ax[0].set_yticks(np.arange(0.5, nfeatures + 0.5), labels=np.arange(nfeatures) + 1)
nspeakers = dump["y"][0].shape[1]
for _ in ax[1:]:
    _.set_ylabel("Speaker #")
    _.set_yticks(np.arange(0.5, nspeakers + 0.5), labels=np.arange(nspeakers) + 1)
fig.savefig("../../Figures/japanese_vowels_test.pdf", dpi=300, bbox_inches="tight")